#### 1. `numeric_columns`

In [ ]:
def numeric_columns(df):
    '''
    Return a list of columns those have numeric values.

    '''
    return df.select_dtypes(include='number').columns

#### 2. Add new rows to an existing DataFrame `df`

```python
df = pd.DataFrame(columns=['col1', 'col2',..., coln])
df.loc[len(df.index)] = [col1_val, col2_val, ..., col_valn]
```

#### 3. Pandas DataFrame to Spark DataFrame

In [ ]:
from pyspark.sql.types import *

def pandas_to_spark(pandas_df):
    # Create equivalent PySpark data types dictionary
    equivalent_types = {
        'datetime64[ns]': TimestampType(),
        'int64': LongType(),
        'int32': IntegerType(),
        'float64': DoubleType(),
        'float32': FloatType()
    }

    # Create StructField based on the column type
    def define_structure(column, format_type):
        spark_type = equivalent_types.get(format_type, StringType())
        return StructField(column, spark_type)

    # Create StructType for the PySpark DataFrame schema
    struct_list = [define_structure(column, typ) for column, typ in pandas_df.dtypes]
    p_schema = StructType(struct_list)

    # Convert Pandas DataFrame to PySpark DataFrame
    return spark.createDataFrame(pandas_df, p_schema)


#### 4. Get week number form a date

In [ ]:
import datetime as dt

def fn_week_number(date):
    def is_leap_year(year):
        return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
    
    day_of_month = date.day()
    days_before_month = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]
    day_of_year = day_of_month + days_before_month[date.month - 1] + (1 if is_leap_year(date.year) and date.month > 2 else 0)
    adjustment = (date.weekday() - day_of_year % 7 + 7) % 7
    return (day_of_year + adjustment) // 7 + 1

date = dt.date(2020, 12, 25)
week_number = fn_week_number(date)
print(week_number)

#### 5. Write to delta
Write the spark DataFrame as a delta table.

```python
df.write.format("delta").mode("overwrite") \
    .option("overwriteSchema",True).save(delta_output_path)
```

#### 6. Remove duplicated columns from a pandas DataFrame while retaining one column

In [ ]:
from collections import Counter
import pandas as pd

def drop_duplicate_cols(df_w_duplicates):
    # Find duplicate columns
    duplicate_cols = [key for key, count in Counter(df_w_duplicates.columns).items() if count > 1]
    print('Number of duplicate columns before dropping: {}'.format(len(duplicate_cols)))

    # Create modified column names
    modified_cols = []
    seen = set()
    for col in df_w_duplicates.columns:
        modified_col = col if col not in seen else '{}_dup'.format(col)
        modified_cols.append(modified_col)
        seen.add(col)

    # Create a DataFrame with modified column names and drop duplicate columns
    drop_cols = [col for col in modified_cols if col.endswith('_dup')]
    drop_df = df_w_duplicates.copy()
    drop_df.columns = modified_cols
    drop_df = drop_df.loc[:, ~drop_df.columns.isin(drop_cols)]

    print('Number of duplicate columns after dropping: {}'.format(len(duplicate_cols(drop_df))))
    return drop_df


#### 7. Supress warnings

In [ ]:
## Globally, for the whole notebook

import warnings

# Option 1: Ignore specific warnings (Ex- FutureWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Option 2: Ignore all warnings
warnings.filterwarnings("ignore")

## Supress warnings for a specific block of code

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    # Warning-causing lines of code here

#### 8. String to Dict

Convert a string in dictionary format to proper python dictionary

In [ ]:
import ast
    
ast.literal_eval("{'muffin' : 'lolz', 'foo' : 'kitty'}")

# This can be applied to pandas df
df['string_col'].apply(lambda x: ast.literal_eval(x) )

#### 9. Reverse Dictionary

```py3
revresed = 
```